# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI. Отдельно проанализируйте 3 ошибочно кластеризованных текста (если такие есть).

In [1]:
import wikipedia
import numpy as np
code2lang = wikipedia.languages()
import nltk
from nltk.tokenize import sent_tokenize
from string import punctuation
from collections import defaultdict
from string import digits
import re
import pandas as pd

In [2]:
langs = ['it', 'de', 'es', 'fr', 'sl']

In [3]:
def load_with_disambigution(page):
    try:
        p = wikipedia.page(page)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        p = wikipedia.page(random_option)
    return p

def get_texts_for_lang(lang, n=100): # функция для скачивания статей из википедии
    # вот так можно зафиксировать язык
    wikipedia.set_lang(lang)
    wiki_content = []
    
    # random достает только ссылки на статьи
    pages = wikipedia.random(n)
    
    for page_name in pages:
        try:
            # чтобы загрузить контент статьи можно вызвать функцию .page
            page = load_with_disambigution(page_name)
        
        except Exception as e:
            print('Skipping page {}'.format(page_name), str(e).strip('\n'))
            continue

        wiki_content.append(f'{page.title}\n{page.content.replace("==", "")}')

    return wiki_content

In [8]:
%%time

wiki_texts = {}

for lang in langs:
    try:
        wiki_texts[lang] = get_texts_for_lang(lang, 10)
    except Exception as e:
        print('ERROR ON - ', lang, str(e).strip('\n'))
        continue
    
    print(lang, len(wiki_texts[lang]))

c:\Users\Alex\scoop\apps\python\current\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Alex\scoop\apps\python\current\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


it 10
Skipping page Therre "Therre" may refer to: 
Hans Therre
Walter Therre
Skipping page Friederike Müller "Friederike Müller" may refer to: 
Friederike Müller (Pianistin)
Friederike Auguste Müller
de 8
es 10
Skipping page Huta-Złomy HTTPSConnectionPool(host='fr.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=Huta-Z%C5%82omy&srinfo=suggestion&format=json&action=query (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001BDE8AEAE10>, 'Connection to fr.wikipedia.org timed out. (connect timeout=None)'))
fr 9
Skipping page Dornier "Dornier" may refer to: 
Claudius Dornier
Claudius Dornier
Dornier Flugzeugwerke
Dornier Flugzeugwerke
DORNIER GmbH
Lindauer DORNIER GmbH
sl 9
CPU times: total: 312 ms
Wall time: 2min 43s


In [9]:
wiki_texts['de'][2]

'Grisebachs Schwertpflanze\nGrisebachs Schwertpflanze (Echinodorus grisebachii), Synonym: E. bleherae, ist eine Pflanzenart aus der Gattung Schwertpflanzen (Echinodorus) innerhalb der Familie der Froschlöffelgewächse (Alismataceae). Sie wird als Aquarienpflanze verwendet.\n\n\n Beschreibung \n\nBei dieser Sumpfpflanze handelt es sich um eine mittel- bis große und kräftig wachsende, ausdauernde krautige Pflanze, die Rhizome ausbildet. Die bandförmigen bis linealischen Laubblätter stehen in einer Blattrosette zusammen. Die Blattoberseite ist mittel- bis dunkelgrün.\n\n\n Taxonomie \nDie Erstbeschreibung von Echinodorus grisebachii erfolgte 1909 durch John Kunkel Small in Nathaniel Lord Britton & al. (Hrsg.): North American Flora, Volume 17, S 46. Das Artepitheton grisebachii ehrt den deutschen Botaniker August Grisebach. Synonyme für Echinodorus grisebachii Small sind: Echinodorus amphibius Rataj, Echinodorus gracilis Rataj, Echinodorus amazonicus Rataj, Echinodorus bleherae Rataj, Echin

In [10]:
stops = punctuation
stops+='\\n—\\t-'
def preproc (text):
    tokenized = nltk.word_tokenize(text)
    prepr_text = ' '.join([token for token in tokenized if token not in stops])
    return prepr_text

In [11]:
preprocessed_wiki = wiki_texts
for lang in preprocessed_wiki:
    i=0
    for text in preprocessed_wiki[lang]:
        prep_text = preproc(text)
        preprocessed_wiki[lang][i] = prep_text
        i+=1

digits = set(digits)

lang2chars = defaultdict (set)


for lang in preprocessed_wiki:
    for text in preprocessed_wiki[lang]:
        char_set = set(re.findall("\w", text.lower())) - digits
        lang2chars[lang].update(char_set)

<>:16: SyntaxWarning: invalid escape sequence '\w'
<>:16: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Alex\AppData\Local\Temp\ipykernel_19652\2889167237.py:16: SyntaxWarning: invalid escape sequence '\w'
  char_set = set(re.findall("\w", text.lower())) - digits


In [12]:
list(lang2chars['fr'])[:5]

['h', 'v', 'b', 'à', 'a']

In [13]:
def predict_language(text, lang2char):
    text_chars = set(re.findall('\w', text.lower())) - digits
    lang2sim = {}
    
    for lang in lang2char:
        intersect = len(text_chars & lang2char[lang])
        lang2sim[lang] = intersect
    
    return max(lang2sim.items(), key=lambda x: x[1])[0]

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Alex\AppData\Local\Temp\ipykernel_19652\3485445251.py:2: SyntaxWarning: invalid escape sequence '\w'
  text_chars = set(re.findall('\w', text.lower())) - digits


In [14]:
wiki_texts['de'][2]

'Grisebachs Schwertpflanze Grisebachs Schwertpflanze Echinodorus grisebachii Synonym E. bleherae ist eine Pflanzenart aus der Gattung Schwertpflanzen Echinodorus innerhalb der Familie der Froschlöffelgewächse Alismataceae Sie wird als Aquarienpflanze verwendet Beschreibung Bei dieser Sumpfpflanze handelt es sich um eine mittel- bis große und kräftig wachsende ausdauernde krautige Pflanze die Rhizome ausbildet Die bandförmigen bis linealischen Laubblätter stehen in einer Blattrosette zusammen Die Blattoberseite ist mittel- bis dunkelgrün Taxonomie Die Erstbeschreibung von Echinodorus grisebachii erfolgte 1909 durch John Kunkel Small in Nathaniel Lord Britton al Hrsg North American Flora Volume 17 S 46 Das Artepitheton grisebachii ehrt den deutschen Botaniker August Grisebach Synonyme für Echinodorus grisebachii Small sind Echinodorus amphibius Rataj Echinodorus gracilis Rataj Echinodorus amazonicus Rataj Echinodorus bleherae Rataj Echinodorus parviflorus Rataj Schwarze Schwertpflanze Ec

In [15]:
sent_tokenized_wiki = wiki_texts
for lang in sent_tokenized_wiki:
    i=0
    for text in sent_tokenized_wiki[lang]:
        if lang == 'de':
            text = sent_tokenize(text, language='german')
        elif lang == 'it':
            text = sent_tokenize(text, language='italian')
        elif lang == 'fr':
            text = sent_tokenize(text, language='french')
        elif lang == 'es':
            text = sent_tokenize(text, language='spanish')
        elif lang == 'sl':
            text = sent_tokenize(text, language='slovene')
        sent_tokenized_wiki[lang][i] = text
        i+=1

In [16]:
sent_tokenized_wiki['de'][2]

['Grisebachs Schwertpflanze Grisebachs Schwertpflanze Echinodorus grisebachii Synonym E. bleherae ist eine Pflanzenart aus der Gattung Schwertpflanzen Echinodorus innerhalb der Familie der Froschlöffelgewächse Alismataceae Sie wird als Aquarienpflanze verwendet Beschreibung Bei dieser Sumpfpflanze handelt es sich um eine mittel- bis große und kräftig wachsende ausdauernde krautige Pflanze die Rhizome ausbildet Die bandförmigen bis linealischen Laubblätter stehen in einer Blattrosette zusammen Die Blattoberseite ist mittel- bis dunkelgrün Taxonomie Die Erstbeschreibung von Echinodorus grisebachii erfolgte 1909 durch John Kunkel Small in Nathaniel Lord Britton al Hrsg North American Flora Volume 17 S 46 Das Artepitheton grisebachii ehrt den deutschen Botaniker August Grisebach Synonyme für Echinodorus grisebachii Small sind Echinodorus amphibius Rataj Echinodorus gracilis Rataj Echinodorus amazonicus Rataj Echinodorus bleherae Rataj Echinodorus parviflorus Rataj Schwarze Schwertpflanze E

In [17]:
data = []
i=1
for lang in wiki_texts:
    for text in wiki_texts[lang]:
        l_id = i
        data.append((text, lang, l_id))
    i+=1

df = pd.DataFrame (data, columns=['sentence','language', 'id'])
print(df)

                                             sentence language  id
0   [Yasmin Finney Yasmin Finney Manchester 30 ago...       it   1
1   [Zorro Zorro Volpe in lingua spagnola è un per...       it   1
2   [Erik Heijblok Erik Heijblok Den Oever 29 giug...       it   1
3   [Ghost Town film 2008 Ghost Town trad lett Cit...       it   1
4   [Kurts Plade Kurts Plade Riga 16 agosto 1898 –...       it   1
5   [Jesmond Delia Jesmond Delia 20 marzo 1967 è u...       it   1
6   [Iuri Capelo Iuri Kaewchang Capelo Macao 27 ot...       it   1
7   [One More Time Rod Stewart One More Time è un ...       it   1
8   [Roy Dotrice Roy Dotrice Guernsey 26 maggio 19...       it   1
9   [Società Autostrade Alto Adriatico Società Aut...       it   1
10  [Pierre Bernard Milius Pierre Bernard Milius 4...       de   2
11  [Liste der Gewässer namens Aa Aa im Alpenraum ...       de   2
12  [Grisebachs Schwertpflanze Grisebachs Schwertp...       de   2
13  [Franz Erasmus Spannheimer Franz Erasmus Spann...       de

In [18]:
import pandas as pd
from sklearn.cluster import AffinityPropagation
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode
from sklearn.metrics import adjusted_rand_score

c:\Users\Alex\scoop\apps\python\current\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Alex\scoop\apps\python\current\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
grouped_df = df.groupby('language')[['sentence', 'language', 'id']]
ARI = []

In [21]:
sentences = df['sentence'].values

texts = list()

for sent in sentences:
    texts.extend(sent)

X = np.zeros((len(texts), 768))
for i, txt in enumerate(texts):
    X[i] = embed(txt)

cluster = AffinityPropagation(damping=0.5)
    
cluster.fit(X)
labels = np.array(cluster.labels_)+1 

sents = list()
true_labels = list()

for id_, sentence_list in zip(df["id"].values, df["sentence"].values):

    sents.extend(sentence_list)
    true_labels.extend([id_]*len(sentence_list))

ARI.append(adjusted_rand_score(true_labels, labels))
    
print(np.mean(ARI)) 


0.2376237623762376


# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор в разных предложениях). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

In [22]:
import bz2
from lxml import etree

with bz2.open('annot.opcorpora.no_ambig_strict.xml.bz2', 'rb') as f_in, open('annot.opcorpora.no_ambig_strict.xml', 'wb') as f_out:
    f_out.write(f_in.read())

open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

In [23]:
sentences = open_corpora.xpath('//tokens')
tokens = sentences[0].xpath('token')

In [24]:
corpus = []


for sentence in open_corpora.xpath('//tokens'):
    sent_tagged = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        sent_tagged.append([word[0]] + gram_info)
    
    corpus.append(sent_tagged)

print(corpus[:10])

[[['«', 'PNCT'], ['Школа', 'NOUN', 'inan', 'femn', 'sing', 'nomn'], ['злословия', 'NOUN', 'inan', 'neut', 'sing', 'gent'], ['»', 'PNCT'], ['учит', 'VERB', 'impf', 'tran', 'sing', '3per', 'pres', 'indc'], ['прикусить', 'INFN', 'perf', 'tran'], ['язык', 'NOUN', 'inan', 'masc', 'sing', 'accs']], [['Сохранится', 'VERB', 'perf', 'intr', 'sing', '3per', 'futr', 'indc'], ['ли', 'PRCL'], ['градус', 'NOUN', 'inan', 'masc', 'sing', 'nomn'], ['дискуссии', 'NOUN', 'inan', 'femn', 'sing', 'gent'], ['в', 'PREP'], ['новом', 'ADJF', 'Qual', 'masc', 'sing', 'loct'], ['сезоне', 'NOUN', 'inan', 'masc', 'sing', 'loct'], ['?', 'PNCT']], [['Великолепная', 'ADJF', 'Qual', 'femn', 'sing', 'nomn'], ['«', 'PNCT'], ['Школа', 'NOUN', 'inan', 'femn', 'sing', 'nomn'], ['злословия', 'NOUN', 'inan', 'neut', 'sing', 'gent'], ['»', 'PNCT'], ['вернулась', 'VERB', 'perf', 'intr', 'femn', 'sing', 'past', 'indc'], ['в', 'PREP'], ['эфир', 'NOUN', 'inan', 'masc', 'sing', 'accs'], ['после', 'PREP'], ['летних', 'ADJF', 'plur',

In [25]:
corp_items = []
corp : dict[str, set[str]] = dict()

for line in corpus:
    for word in line:
        name = word[0]
        gram = str(word[1:])
        if name not in corp_items:
            corp[name] = {gram}
            corp_items.append(name)
        else:
            corp[name].add(gram)

In [26]:
print (corp['США'])

{"['NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'ablt']", "['NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'loct']", "['NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'accs']", "['NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'nomn']", "['NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'gent']", "['NOUN', 'inan', 'GNdr', 'Pltm', 'Fixd', 'Abbr', 'Geox', 'plur', 'datv']"}


In [27]:
from collections import Counter

In [28]:
corp_most_items = []

for line in corpus:
    for word in line:
        name = word[0]
        corp_most_items.append(name)

In [29]:
counter_omonims = Counter()

for word, gram in corp.items():
    if len(gram)>1:
        num_words = corp_most_items.count(word)
        counter_omonims[word] = num_words

counter_omonims.most_common(5) #самые частотные морфологически омонимичные словоформы

[('в', 1583), ('на', 686), ('с', 536), ('и', 507), ('году', 115)]

In [30]:
counter_longest = Counter()

for word, gram in corp.items():
    if len(gram)>1:
        counter_longest[word] = len(gram)

counter_longest.most_common(5) #словоформы с самым большим количеством вариантов грам. разбора

[('США', 6), ('кино', 5), ('евро', 5), ('компании', 5), ('какой', 5)]

## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуры через DependencyGraph, выберите 3 любых отношения и для каждого найдите топ-5 самых встречаемых пар слов, связанных этим отношением. 

Для самой частотной пары слов в каждом из отношений вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)` и сортируя результат по порядку слов в предложениях, аналогично тому как я делал с summaries только у вас будет два слова) 
В итоге у вас должен получится что-то такое:

```
### отношение
relation_name

### топ 5 пар слов связанных этим отношением
(word1, word2), (word3, word4), (word5, word6), (word7, word8), (word9, word10)

### подзависимые для самого частотного
(subword word1 subword, word2 subword subword)

... (и так три раза)
```


In [31]:
from nltk.parse import DependencyGraph
import warnings
import os
warnings.filterwarnings('ignore')

In [32]:
file = open('ru_syntagrus-ud-train-b.conllu', encoding='utf-8').read()

In [33]:
trees = []
parsed_sents = [sentence.split("\n") for sentence in file.split('\n\n')]
for sentence in parsed_sents:
    tree = [x.replace("\t", " ") for x in sentence if not x.startswith("#")]
    trees.append('\n'.join(tree))

In [34]:
def flatten_list(l):
    result = []
    
    for el in l:
        # окончание рекурсии - если текущий элемент не список
        if not isinstance(el, list):
            result.append(el)
        #если текущий элемент список, то применяем к нему flatten_list 
        else:
            result += flatten_list(el)
    
    return result

def get_subtree(nodes, node):
    
    
    if not nodes[node]['deps']:
        return [node]
    
    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] 
                         if rel != 'punct'
                         for dep in nodes[node]['deps'][rel]]



Отношение 'NOUN' - 'appos'

In [35]:
pairs_appos = Counter()

AAA = []  #сюда пойдут битые предложения, потому что DependencyGraph их почему-то не воспринимает :(

for tree in trees:
    try:
        d = DependencyGraph(tree)
        d.root = d.nodes[0] 
        triples = list(d.triples())
        for e1, rel, e2 in triples:
            if e1[1] == 'NOUN' and rel == 'appos':
                pairs_appos[e1[0], e2[0]]+=1
    except Exception:
        AAA.append(tree)
        pass
counter_appos = Counter(pairs_appos)
counter_appos.most_common(5)

[(('тетя', 'Шура'), 12),
 (('президент', 'Владимир'), 10),
 (('г-н', 'Путин'), 8),
 (('провинции', 'Пактия'), 8),
 (('движения', 'Талибан'), 6)]

In [36]:
w1_appos, w2_appos = counter_appos.most_common(1)[0][0]
subtrees_for_appos1 = list()
subtrees_for_appos2 = list()
for tree in trees:
    try:
        d = DependencyGraph(tree)
        d.root = d.nodes[0] 
        for i, node in d.nodes.items():
            if node['word'] == w1_appos:
                index1_appos = i
                appos1 = (sorted(flatten_list(get_subtree(d.nodes, index1_appos))))
                for j in appos1:
                    subtrees_for_appos1.append(d.nodes[j]['word'])
            elif node['word'] == w2_appos:
                index2_appos = i
                appos2 = (sorted(flatten_list(get_subtree(d.nodes, index2_appos))))
                for j in appos2: 
                    subtrees_for_appos2.append(d.nodes[j]['word'])
    except Exception:
        pass


In [37]:
print('Тип отношений - appos')
print('Топ 5 пар слов связанных этим отношением: ', counter_appos.most_common(5))
print('Подзависимые для самого частотного:\n',w1_appos, ' - ', subtrees_for_appos1[:20], '\n', w2_appos, ' - ', subtrees_for_appos2[:20])

Тип отношений - appos
Топ 5 пар слов связанных этим отношением:  [(('тетя', 'Шура'), 12), (('президент', 'Владимир'), 10), (('г-н', 'Путин'), 8), (('провинции', 'Пактия'), 8), (('движения', 'Талибан'), 6)]
Подзависимые для самого частотного:
 тетя  -  ['ее', 'тетя', 'у', 'которой', 'она', 'жила', 'Ее', 'тетя', 'вдова', 'священника', 'Шурочкина', 'тетя', 'моя', 'любимая', 'тетя', 'Клава', 'тетя', 'тетя', 'Вера', 'и'] 
 Шура  -  ['Шура', 'Шура', 'Шура', 'Шура', 'Шура', 'Шура', 'управдомша', 'Шура', 'Шура', 'управдомша', 'Шура', 'Шура', 'управдомша', 'Шура', 'Шура', 'управдомша', 'Шура', 'Шура', 'Шура', 'Шура']


NOUN - 'case'

In [38]:
pairs_case = Counter()

for tree in trees:
    try:
        d = DependencyGraph(tree)
        d.root = d.nodes[0] 
        triples = list(d.triples())
        for e1, rel, e2 in triples:
            if e1[1] == 'NOUN' and rel=='case':
                 pairs_case[e2[0], e1[0]]+=1
    except Exception:
        pass

counter_case = Counter(pairs_case)
counter_case.most_common(5)

[(('в', 'году'), 269),
 (('в', 'случае'), 132),
 (('По', 'словам'), 121),
 (('в', 'время'), 117),
 (('в', 'числе'), 96)]

In [39]:
w1_case, w2_case = counter_case.most_common(1)[0][0]
subtrees_for_case1 = list()
subtrees_for_case2 = list()
for tree in trees:
    try:
        d = DependencyGraph(tree)
        d.root = d.nodes[0] 
        for i, node in d.nodes.items():
            if node['word'] == w1_case:
                index1_case = i
                case1 = (sorted(flatten_list(get_subtree(d.nodes, index1_case))))
                for j in case1:
                    subtrees_for_case1.append(d.nodes[j]['word'])
            elif node['word'] == w2_case:
                index2_case = i
                case2 = (sorted(flatten_list(get_subtree(d.nodes, index2_case))))
                for j in case2: 
                    subtrees_for_case2.append(d.nodes[j]['word'])
    except Exception:
        pass

In [40]:
print('Тип отношений - case')
print('Топ 5 пар слов связанных этим отношением: ', counter_case.most_common(5))
print('Подзависимые для самого частотного:\n',w1_case, ' - ', subtrees_for_case1[:20], '\n', w2_case, ' - ', subtrees_for_case2[:20])

Тип отношений - case
Топ 5 пар слов связанных этим отношением:  [(('в', 'году'), 269), (('в', 'случае'), 132), (('По', 'словам'), 121), (('в', 'время'), 117), (('в', 'числе'), 96)]
Подзависимые для самого частотного:
 в  -  ['в', 'в', 'в', 'в', 'в', 'в', 'в', 'в', 'в', 'частности', 'в', 'в', 'в', 'в', 'связи', 'с', 'в', 'в', 'в', 'в'] 
 году  -  ['В', '2007', '2008', 'учебном', 'году', 'В', 'этом', 'году', 'В', 'этом', 'году', 'В', '2008', 'году', 'в', 'прошлом', 'году', 'уже', 'в', 'этом']


NOUN - 'nmod'

In [41]:
pairs_nmod = Counter()

for tree in trees:
    try:
        d = DependencyGraph(tree)
        d.root = d.nodes[0] 
        triples = list(d.triples())
        for e1, rel, e2 in triples:
            if e1[1] == 'NOUN' and rel=='nmod':
                 pairs_nmod[e1[0], e2[0]]+=1
    except Exception:
        pass

counter_nmod = Counter(pairs_nmod)
counter_nmod.most_common(5)

[(('начале', 'века'), 26),
 (('корреспондент', 'РИА'), 23),
 (('президента', 'России'), 17),
 (('правам', 'человека'), 12),
 (('зависимости', 'того'), 11)]

In [42]:
w1_nmod, w2_nmod = counter_nmod.most_common(1)[0][0]
subtrees_for_nmod1 = list()
subtrees_for_nmod2 = list()
for tree in trees:
    try:
        d = DependencyGraph(tree)
        d.root = d.nodes[0] 
        for i, node in d.nodes.items():
            if node['word'] == w1_nmod:
                index1_nmod = i
                nmod1 = (sorted(flatten_list(get_subtree(d.nodes, index1_nmod))))
                for j in nmod1:
                    subtrees_for_nmod1.append(d.nodes[j]['word'])
            elif node['word'] == w2_nmod:
                index2_nmod = i
                nmod2 = (sorted(flatten_list(get_subtree(d.nodes, index2_nmod))))
                for j in nmod2: 
                    subtrees_for_nmod2.append(d.nodes[j]['word'])
    except Exception:
        pass


In [43]:
print('Тип отношений - nmod')
print('Топ 5 пар слов связанных этим отношением: ', counter_nmod.most_common(5))
print('Подзависимые для самого частотного:\n',w1_nmod, ' - ', subtrees_for_nmod1[:20], '\n', w2_nmod, ' - ', subtrees_for_nmod2[:20])

Тип отношений - nmod
Топ 5 пар слов связанных этим отношением:  [(('начале', 'века'), 26), (('корреспондент', 'РИА'), 23), (('президента', 'России'), 17), (('правам', 'человека'), 12), (('зависимости', 'того'), 11)]
Подзависимые для самого частотного:
 начале  -  ['в', 'начале', '90-х', 'годов', 'прошлого', 'века', 'в', 'начале', 'прошлого', 'года', 'в', 'начале', '2000-х', 'в', 'начале', 'следующего', 'В', 'начале', '2000-х', 'В'] 
 века  -  ['прошлого', 'века', 'почти', 'два', 'века', 'и', 'исторические', 'пробы', 'и', 'ошибки', 'якобинский', 'террор', 'реставрация', 'монархии', 'две', 'империи', 'и', 'множество', 'попыток', 'установления']
